<h1> <b>Homework 1</b></h1>
<i>completed by Alejandro J. Rojas</i>

<h2>HW1.0.0.</h2> Define big data. Provide an example of a big data problem in your domain of expertise. 

<h2>HW1.0.1.</h2>In 500 words (English or pseudo code or a combination) describe how to estimate the bias, the variance, the irreduciable error for a test dataset T when using polynomial regression models of degree 1, 2,3, 4,5 are considered. How would you select a model?

<h2> HW1.1.</h2> Read through the provided control script (pNaiveBayes.sh)
   and all of its comments. When you are comfortable with their
   purpose and function, respond to the remaining homework questions below. 
   A simple cell in the notebook with a print statmement with  a "done" string will suffice here. (dont forget to include the Question Number and the quesition in the cell as a multiline comment!)

In [ ]:
#########  Done  #########

<h2>HW1.6</h2> Benchmark your code with the Python SciKit-Learn implementation of multinomial Naive Bayes

<h2>HW1.2.</h2>Provide a mapper/reducer pair that, when executed by pNaiveBayes.sh
   will determine the number of occurrences of a single, user-specified word. Examine the word “assistance” and report your results.

In [34]:
#Map

In [136]:
%%writefile mapper.py
#!/usr/bin/python
## mapper.py
## Author: Alejandro J. Rojas
## Description: mapper code for HW1.2-1.5

import sys
import re
count = 0
records = 0
words = 0

## collect user input
filename = sys.argv[1]
findwords = re.split(" ",sys.argv[2].lower())

with open (filename, "r") as myfile:
#We set to look for the word on each line assuming different ways in which it could be present
    for line in myfile.readlines():
        record = re.split(r'\t+', line)
        records = records + 1
        for i in range (len(record)):
            bagofwords = re.split(" ",record[i])
            for word in bagofwords:
                words = words + 1
                for keyword in findwords:
                    if keyword in word:
                        count = count + 1
            

##print '# of Records analized',records
##print '# of Words analized', words
##print '# of Ocurrences', count
print count

Overwriting mapper.py


In [137]:
!chmod +x mapper.py

In [126]:
#Reduce

In [217]:
%%writefile reducer.py
#!/usr/bin/python
## reducer.py
## Author: Alejandro J. Rojas
## Description: reducer code for HW1.2-1.5

import sys
import re
sum = 0

## collect user input
filenames = sys.argv[1:]
for file in filenames:
    with open (file, "r") as myfile:
        for line in myfile.readlines():
            if line.strip():
                sum = sum + int(line)
                
print sum

Overwriting reducer.py


In [198]:
!chmod +x reducer.py

# Write script to file

In [350]:
%%writefile pNaiveBayes.sh
## pNaiveBayes.sh
## Author: Jake Ryland Williams
## Usage: pNaiveBayes.sh m wordlist
## Input:
##       m = number of processes (maps), e.g., 4
##       wordlist = a space-separated list of words in quotes, e.g., "the and of"
##
## Instructions: Read this script and its comments closely.
##               Do your best to understand the purpose of each command,
##               and focus on how arguments are supplied to mapper.py/reducer.py,
##               as this will determine how the python scripts take input.
##               When you are comfortable with the unix code below,
##               answer the questions on the LMS for HW1 about the starter code.

## collect user input
m=$1 ## the number of parallel processes (maps) to run
wordlist=$2 ## if set to "*", then all words are used

## a test set data of 100 messages
data="enronemail_1h.txt" 

## the full set of data (33746 messages)
# data="enronemail.txt" 

## 'wc' determines the number of lines in the data
## 'perl -pe' regex strips the piped wc output to a number
linesindata=`wc -l $data | perl -pe 's/^.*?(\d+).*?$/$1/'`

## determine the lines per chunk for the desired number of processes
linesinchunk=`echo "$linesindata/$m+1" | bc`

## split the original file into chunks by line
split -l $linesinchunk $data $data.chunk.

## assign python mappers (mapper.py) to the chunks of data
## and emit their output to temporary files
for datachunk in $data.chunk.*; do
    ## feed word list to the python mapper here and redirect STDOUT to a temporary file on disk
    ####
    ####
    ./mapper.py $datachunk "$wordlist" > $datachunk.counts &
    ####
    ####
done
## wait for the mappers to finish their work
wait

## 'ls' makes a list of the temporary count files
## 'perl -pe' regex replaces line breaks with spaces
countfiles=`\ls $data.chunk.*.counts | perl -pe 's/\n/ /'`

## feed the list of countfiles to the python reducer and redirect STDOUT to disk
####
####
./reducer.py $countfiles > $data.output
####
####
numOfInstances=$(cat $data.output)
echo "found [$numOfInstances] [$wordlist]" ## Report  how many were found
## clean up the data chunks and temporary count files
\rm $data.chunk.*

Overwriting pNaiveBayes.sh


In [285]:
!chmod a+x pNaiveBayes.sh

In [209]:
# Run file

Usage: usage: pNaiveBayes.sh m wordlist

In [210]:
!./pNaiveBayes.sh 5 "assistance"

found [10] [assistance]


<h2>HW1.3.</h2> Provide a mapper/reducer pair that, when executed by pNaiveBayes.sh
   will classify the email messages by a single, user-specified word using the multinomial Naive Bayes Formulation. Examine the word “assistance” and report your results. 

In [48]:
#Map

In [67]:
%%writefile mapper.py
#!/usr/bin/python
## mapper.py
## Author: Alejandro J. Rojas
## Description: mapper code for HW1.3

import sys
import re



########## Collect user input  ###############
filename = sys.argv[1]
findwords = re.split(" ",sys.argv[2].lower())




with open (filename, "r") as myfile:

    for line in myfile.readlines():
        record = re.split(r'\t+', line)                    ### Each email is a record with 4 components
                                                           ### 1) ID 2) Spam Truth 3) Subject 4) Content
        if len(record)==4:                                 ### Take only complete records

            ########## Variables to collect and measure #########
            records = 0                                    ### Each record corresponds to a unique email
            words = 0                                      ### Words written in all emails incluidng Subject 
            spam_records, spam_words, spam_count = 0,0,0   ### Spam email count, words in spam email, user-specified word count
            ham_records, ham_words, ham_count = 0, 0, 0    ### Same as above but for not spam emails


            records += 1                                   ### add one the the total sum of emails
            if int(record[1]) == 1:                        ### If the email is labeled as spam
                spam_records += 1                          ### add one to the email spam count
                for i in range (2,len(record)):            ### Starting from Subject to the Content               
                    bagofwords = re.split(" ",record[i])   ### Collect all words present on each email                
                    for word in bagofwords:                ### For each word
                        words += 1                         ### add one to the total sum of words
                        spam_words += 1                    ### add one to the total sum of spam words  
                        for keyword in findwords:          ### for each word specified by user
                            if keyword in word:            ### If there's a match then
                                spam_count += 1            ### add one to the user specified word count as spam
                                
            else:                                          ### If email is not labeled as spam
                ham_records +=1                            ### add one to the email ham count
                for i in range (2,len(record)):            ### Starting from Subject to the Content               
                    bagofwords = re.split(" ",record[i])   ### Collect all words present on each email                
                    for word in bagofwords:                ### For each word
                        words += 1                         ### add one to the total sum of words
                        ham_words += 1                     ### add one to the total sum of ham words  
                        for keyword in findwords:          ### for each word specified by user
                            if keyword in word:            ### If there's a match then
                                ham_count += 1             ### add one to the user specified word count as ham
                    
            record_id = record[0]
            truth = record[1]
            print spam_count, " ", spam_words, " ",  spam_records, " ",  \
                  ham_count, " ", ham_words, " ",  ham_records, " ", \
                  words, " ", records, " ",  record_id, " ", truth      

Overwriting mapper.py


In [68]:
!chmod +x mapper.py

In [69]:
%%writefile reducer.py
#!/usr/bin/python
## reducer.py
## Author: Alejandro J. Rojas
## Description: reducer code for HW1.3-1.4

import sys
import re
sum_spam_records, sum_spam_words, sum_spam_count = 0,0,0
sum_ham_records, sum_ham_words, sum_ham_count = 0,0,0
sum_records,sum_words = 0,0

## collect user input
filenames = sys.argv[1:]
for file in filenames:
    with open (file, "r") as myfile:
        for line in myfile.readlines():
            if line.strip():
                factors = re.split(" ", line)
                sum_spam_count += int(factors[0])           ## sum up every time the word was found in a spam
                sum_spam_words += int(factors[3])           ## sum up all words from spams
                sum_spam_records+= int(factors[6])          ## sum up all emails labeled as spam
                sum_ham_count  += int(factors[9])           ## sum up every time the word was found in a ham
                sum_ham_words += int(factors[12])           ## sum up all words from hams
                sum_ham_records += int(factors[15])         ## sum up all emails labeled as ham
                sum_words += int(factors[18])               ## sum all words from all emails
                sum_records += int(factors[21])             ## sum all emails
                

prior_spam = float(sum_spam_records)/float(sum_records)     ## prior prob of a spam email
prior_ham = float(sum_ham_records)/float(sum_records)       ## prior prob of a ham email
prob_word_spam = float(sum_spam_count)/float(sum_spam_words)## prob of word given that email is spam
prob_word_ham = float(sum_ham_count)/float(sum_ham_words)   ## prob of word given that email is ham

##check_prior = prior_spam + prior_ham                        ## check priors -> sum to 1
##check_words = float(sum_words)/float(sum_spam_words+sum_ham_words) ## check probabilities of a word -> sum to 1
##check_spam = prob_word_spam*float(sum_spam_words)/float(sum_spam_count) ## check spam counts -> sum to 1
##check_ham = prob_word_ham*float(sum_ham_words)/float(sum_ham_count) ## check ham count -> sum to 1
sum_count = sum_spam_count+sum_ham_count

print "Summary of Data"
print '%4s'%sum_records ,'emails examined, containing %6s'%sum_words, 'words, we found %3s'%sum_count ,'matches.'

print '%30s' %'ID', '%10s' %'TRUTH', '%10s' %'CLASS'
for file in filenames:                                      
    with open (file, "r") as myfile:
        for line in myfile.readlines():
            if line.strip():
                data = re.split(" ", line)
                record_id = data[24]
                y_true = data[27][0]
                count = int(data[0]) + int(data[9])
                p_spam = prior_spam*prob_word_spam**count
                p_ham = prior_ham*prob_word_ham**count
                if p_spam > p_ham:
                    y_pred = 1
                else:
                    y_pred = 0
                    
                print  '%30s' %record_id, '%10s' %y_true, '%10s' %y_pred
                
             

Overwriting reducer.py


In [70]:
!chmod +x reducer.py

In [71]:
%%writefile pNaiveBayes.sh
## pNaiveBayes.sh
## Author: Jake Ryland Williams
## Usage: pNaiveBayes.sh m wordlist
## Input:
##       m = number of processes (maps), e.g., 4
##       wordlist = a space-separated list of words in quotes, e.g., "the and of"
##
## Instructions: Read this script and its comments closely.
##               Do your best to understand the purpose of each command,
##               and focus on how arguments are supplied to mapper.py/reducer.py,
##               as this will determine how the python scripts take input.
##               When you are comfortable with the unix code below,
##               answer the questions on the LMS for HW1 about the starter code.

## collect user input
m=$1 ## the number of parallel processes (maps) to run
wordlist=$2 ## if set to "*", then all words are used

## a test set data of 100 messages
data="enronemail_1h.txt" 

## the full set of data (33746 messages)
# data="enronemail.txt" 

## 'wc' determines the number of lines in the data
## 'perl -pe' regex strips the piped wc output to a number
linesindata=`wc -l $data | perl -pe 's/^.*?(\d+).*?$/$1/'`

## determine the lines per chunk for the desired number of processes
linesinchunk=`echo "$linesindata/$m+1" | bc`

## split the original file into chunks by line
split -l $linesinchunk $data $data.chunk.

## assign python mappers (mapper.py) to the chunks of data
## and emit their output to temporary files
for datachunk in $data.chunk.*; do
    ## feed word list to the python mapper here and redirect STDOUT to a temporary file on disk
    ####
    ####
    ./mapper.py $datachunk "$wordlist" > $datachunk.counts &
    ####
    ####
done
## wait for the mappers to finish their work
wait

## 'ls' makes a list of the temporary count files
## 'perl -pe' regex replaces line breaks with spaces
countfiles=`\ls $data.chunk.*.counts | perl -pe 's/\n/ /'`

## feed the list of countfiles to the python reducer and redirect STDOUT to disk
####
####
./reducer.py $countfiles > $data.output
####
####
numOfInstances=$(cat $data.output)
echo "NB Classifier based on word(s): $wordlist" ## Print out words 
echo "$numOfInstances" ## Print out output data
## clean up the data chunks and temporary count files
\rm $data.chunk.*

Overwriting pNaiveBayes.sh


In [72]:
!./pNaiveBayes.sh 5 "assistance"

NB Classifier based on word(s): assistance
Summary of Data
  98 emails examined, containing  35352 words, we found   9 matches.
                            ID      TRUTH      CLASS
        0001.1999-12-10.farmer          0          0
      0001.1999-12-10.kaminski          0          0
          0001.2000-01-17.beck          0          0
       0001.2001-02-07.kitchen          0          0
      0001.2001-04-02.williams          0          0
        0002.1999-12-13.farmer          0          0
       0002.2001-02-07.kitchen          0          0
     0002.2001-05-25.SA_and_HP          1          0
            0002.2003-12-18.GP          1          0
            0002.2004-08-01.BG          1          1
      0003.1999-12-10.kaminski          0          0
        0003.1999-12-14.farmer          0          0
          0003.2000-01-17.beck          0          0
       0003.2001-02-08.kitchen          0          0
            0003.2003-12-18.GP          1          0
            0003.2004-08

<h2>HW1.4.</h2> Provide a mapper/reducer pair that, when executed by pNaiveBayes.sh
   will classify the email messages by a list of one or more user-specified words. Examine the words “assistance”, “valium”, and “enlargementWithATypo” and report your results

In [507]:
!./pNaiveBayes.sh 5 "assistance valium enlargementWithATypo"

NB Classifier based on word(s): assistance valium enlargementWithATypo
Summary of Data
  98 emails examined, containing   35352 words, we found   12 matches.
                            ID      TRUTH      CLASS
        0001.1999-12-10.farmer          0          0
      0001.1999-12-10.kaminski          0          0
          0001.2000-01-17.beck          0          0
       0001.2001-02-07.kitchen          0          0
      0001.2001-04-02.williams          0          0
        0002.1999-12-13.farmer          0          0
       0002.2001-02-07.kitchen          0          0
     0002.2001-05-25.SA_and_HP          1          0
            0002.2003-12-18.GP          1          0
            0002.2004-08-01.BG          1          1
      0003.1999-12-10.kaminski          0          0
        0003.1999-12-14.farmer          0          0
          0003.2000-01-17.beck          0          0
       0003.2001-02-08.kitchen          0          0
            0003.2003-12-18.GP          1      

<h2>HW1.5.</h2> Provide a mapper/reducer pair that, when executed by pNaiveBayes.sh
   will classify the email messages by all words present.

In [34]:
#Map

In [1]:
%%writefile mapper.py
#!/usr/bin/python
## mapper.py
## Author: Alejandro J. Rojas
## Description: mapper code for HW1.5

import sys
import re



########## Collect user input  ###############
filename = sys.argv[1]
findwords = []
countwords = []


def line_preprocessor(s):
    elim_array = ['\s,\s', 
                 '\s&\s', 
                 '\s"\s', 
                  '\s|\s', 
                '\s\\s',
                  '\s?\s', 
                  ' \s-\s', 
                  '\s!\s', 
                 '\d'
                  ]
    for elim_word in  elim_array:    
        s = re.sub(elim_word,'', s, flags=re.IGNORECASE)
    return s

with open (filename, "r") as myfile:

    for line in myfile.readlines():
        record = line_preprocessor(line)                   ### Preprocess line
        record = re.split(r'\t+', line)                    ### Each email is a record with 4 components
                                                           ### 1) ID 2) Spam Truth 3) Subject 4) Content
        if len(record)==4:                                 ### Take only complete records

            for i in range (2,len(record)):                ### Starting from Subject to the Content               
                bagofwords = re.split(" ",record[i])       ### Collect all words present on each email               
                for word in bagofwords:                
                    findwords.append(word)                 ### Add each word to list
    findwords = sorted(set(findwords))                     ### Eliminate repeats and sort

with open (filename, "r") as myfile:    
    for line in myfile.readlines():
        record = re.split(r'\t+', line)                    ### Each email is a record with 4 components
                                                           ### 1) ID 2) Spam Truth 3) Subject 4) Content
        if len(record)==4:                                 ### Take only complete records

            record_wordlist =[]                            ### initiate a list of words that show up in email
            for i in range (2,len(record)):                ### Starting from Subject to the Content               
                bagofwords = re.split(" ",record[i])       ### Collect all words present on each email               
                for word in bagofwords:
                    record_wordlist.append(word)           ### place each word on the list
            
            for keyword in findwords:                     
                count = 0        
                for word in record_wordlist:               ### review each word in the email
                    if keyword in word:                    ### add one every time we find a word in the email
                        count += 1
                    print record[0], '\t', record[1],'\t', word,'\t',count ### print out each word tuple 
                    
            



Overwriting mapper.py


In [2]:
!chmod +x mapper.py

In [3]:
#Reduce

In [4]:
%%writefile reducer.py
#!/usr/bin/python
## reducer.py
## Author: Alejandro J. Rojas
## Description: reducer code for HW1.5

import sys
import re

word_tuple, id_list, word_list = [], [], []
sum_spam_records, sum_ham_records, sum_records = 0,0,0
sum_spam_words, sum_ham_words, sum_words = 0,0,0
wordcount_spam, wordcount_ham, wordcount = [], [], []



## collect user input
filenames = sys.argv[1:]
for file in filenames:
    with open (file, "r") as myfile:
        for line in myfile.readlines():
            if line.strip():
                data = re.split(r'\t+', line)
                word_tuple.append((data[0],data[1],data[2],data[3]))
                id_list.append(data[0])
                word_list.append(data[2])
                sum_words += int(data[3])
                
unique_ids = sorted(set(id_list))
vocabulary = sorted(set(word_list))
for email in unique_ids:
    for i in range(len(word_tuple)):
        if email == word_tuple[i][0]:
            y_true = { 'email': word_tuple[i][0],
                      'truth': word_tuple[i][1]
                      }
    y_true_list.append(y_true) 

for i in range(len(word_tuple)):
    if word_tuple[i][1] == 1:
        spam_ids.append(word_tuple[i][0])
        sum_spam_words += word_tuple[i][3]
    else:
        ham_ids.append(word_tuple[i][0])
        sum_ham_words += word_tuple[i][3]

sum_spam_records = len(set(spam_ids))
sum_ham_records = len(set(ham_ids))
sum_records = len(unique_ids)

prior_spam = float(sum_spam_records)/float(sum_records)     ## prior prob of a spam email
prior_ham = float(sum_ham_records)/float(sum_records)       ## prior prob of a ham email

for word in vocabulary:
    count, spam_count, ham_count = 0,0,0
    for i in range(len(word_tuple)):
        if word == word_tuple[i][2]:
            count+= word_tuple[i][3]
            if word_tuple[i][1] == 1:
                spam_count += 1
            else:
                ham_count += 1
    
    
    
    wordcount.append(count)
    wordcount_spam.append(spam_count)
    wordcount_ham.append(ham_count)
    sum_words = sum(wordcount)
    sum_spam_words = sum(wordcount_spam)
    sum_ham_word =sum(wordcount_ham)

for email in unique_ids:
    wordcount_in_email =[]
    for word in vocabulary:
        found = False
        for i in range(len(word_tuple)):
            if email == word_tuple[i][0]:            
                if word == word_tuple[i][2] :
                    wordcount_in_email.append(word_tuple[i][3])
                    found = True
        if found == False:
             wordcount_in_email.append(0)
    
    for i in range(len(vocabulary)):
        cond_prob_spam += float(wordcount_spam[i])/float(sum_spam_words)
        cond_prob_ham += float(wordcount_ham[i])/float(sum_ham_words)
        p_spam += prior_spam*cond_prob_spam**float(wordcount_in_email[i])
        p_ham += prior_ham*cond_prob_ham**float(wordcount_in_email[i])
    
    if p_spam > p_ham:
        y_pred = 1
    else:
        y_pred = 0
    


##check_prior = prior_spam + prior_ham                        ## check priors -> sum to 1
##check_words = float(sum_words)/float(sum_spam_words+sum_ham_words) ## check probabilities of a word -> sum to 1
##check_spam = prob_word_spam*float(sum_spam_words)/float(sum_spam_count) ## check spam counts -> sum to 1
##check_ham = prob_word_ham*float(sum_ham_words)/float(sum_ham_count) ## check ham count -> sum to 1
##print word_tuple

print "Summary of Data"
print '%4s'%sum_records ,'emails examined, containing %6s'%sum_words, 'using a dictionary of %6s' %size_dict

print '%30s' %'ID', '%10s' %'TRUTH', '%10s' %'CLASS'
for file in filenames:                                      
    with open (file, "r") as myfile:
        for line in myfile.readlines():
            if line.strip():
                data = re.split(" - ", line)
                record_id = data[0]
                y_true = data[1][0]
                count_vector = data[2]
                cond_prob_spam = []
                cond_prob_ham = []
                for i in range (len(wordcount)):
                    prob_spam = float((count_vector[i]+1))/float(word_count_spam[i])**int(wordcount[i])
                    prob_ham = float((count_vector[i]+1))/float(word_count_ham[i])**int(wordcount[i])
                    p_spam += prior_spam*prob_spam
                    p_ham += prior_ham*prob_ham
                   
                if p_spam > p_ham:
                    y_pred = 1
                else:
                    y_pred = 0
                    
                print  '%30s' %record_id, '%10s' %y_true, '%10s' %y_pred
                
             

Overwriting reducer.py


In [5]:
!chmod +x reducer.py

In [6]:
%%writefile pNaiveBayes.sh
## pNaiveBayes.sh
## Author: Jake Ryland Williams
## Usage: pNaiveBayes.sh m wordlist
## Input:
##       m = number of processes (maps), e.g., 4
##       wordlist = a space-separated list of words in quotes, e.g., "the and of"
##
## Instructions: Read this script and its comments closely.
##               Do your best to understand the purpose of each command,
##               and focus on how arguments are supplied to mapper.py/reducer.py,
##               as this will determine how the python scripts take input.
##               When you are comfortable with the unix code below,
##               answer the questions on the LMS for HW1 about the starter code.

## collect user input
m=$1 ## the number of parallel processes (maps) to run

## a test set data of 100 messages
data="enronemail_1h.txt" 

## the full set of data (33746 messages)
# data="enronemail.txt" 

## 'wc' determines the number of lines in the data
## 'perl -pe' regex strips the piped wc output to a number
linesindata=`wc -l $data | perl -pe 's/^.*?(\d+).*?$/$1/'`

## determine the lines per chunk for the desired number of processes
linesinchunk=`echo "$linesindata/$m+1" | bc`

## split the original file into chunks by line
split -l $linesinchunk $data $data.chunk.

## assign python mappers (mapper.py) to the chunks of data
## and emit their output to temporary files
for datachunk in $data.chunk.*; do
    ## feed word list to the python mapper here and redirect STDOUT to a temporary file on disk
    ####
    ####
    ./mapper.py $datachunk > $datachunk.counts &
    ####
    ####
done
## wait for the mappers to finish their work
wait

## 'ls' makes a list of the temporary count files
## 'perl -pe' regex replaces line breaks with spaces
countfiles=`\ls $data.chunk.*.counts | perl -pe 's/\n/ /'`

## feed the list of countfiles to the python reducer and redirect STDOUT to disk
####
####
./reducer.py $countfiles > $data.output
####
####
numOfInstances=$(cat $data.output)
echo "NB Classifier" ## Print out words 
echo "$numOfInstances" ## Print out output data
## clean up the data chunks and temporary count files
##\rm $data.chunk.*

Overwriting pNaiveBayes.sh


In [7]:
!./pNaiveBayes.sh 5

KeyboardInterrupt: 